# PROGRAMA DE CIENCIAS DE LOS DATOS 
## **CURSO: BIG DATA**
### **TAREA #5**

Profesor: MSc. Felipe Meza


Alumnos: 

  Lester Salazar Viales
  
  Randal Salazar Viales

# Intro ML SUP en BD

Creación de la sesión Spark:

In [1]:
#import SparkSession

from pyspark.sql import SparkSession

In [2]:
# Crear el spark session object, llamarle "supervised_ml"

spark=SparkSession.builder.appName('supervised_ml').getOrCreate()

## Regression 

Carga de datos, archivo *Linear_regression_dataset.csv*:

In [3]:
# Carga de datos

df=spark.read.csv('Linear_regression_dataset.csv',inferSchema=True,header=True)

Se invocan las librerias correspondientes a **LinearRegression**, asi como las de OneHotEncoder, StringIndexer, VectorAssembler:

In [4]:
# Importacion de libs y operaciones

from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler

from pyspark.ml.regression import LinearRegression

Se visualizan algunos datos:

In [5]:
print((df.count(), len(df.columns))) 

(1232, 6)


Se muestran los primeros 10 datos:

In [6]:
# primeros 10 datos

df.show(10)

+-----+-----+-----+-----+-----+-----+
|var_1|var_2|var_3|var_4|var_5|label|
+-----+-----+-----+-----+-----+-----+
|  734|  688|   81|0.328|0.259|0.418|
|  700|  600|   94| 0.32|0.247|0.389|
|  712|  705|   93|0.311|0.247|0.417|
|  734|  806|   69|0.315| 0.26|0.415|
|  613|  759|   61|0.302| 0.24|0.378|
|  748|  676|   85|0.318|0.255|0.422|
|  669|  588|   97|0.315|0.251|0.411|
|  667|  845|   68|0.324|0.251|0.381|
|  758|  890|   64| 0.33|0.274|0.436|
|  726|  670|   88|0.335|0.268|0.422|
+-----+-----+-----+-----+-----+-----+
only showing top 10 rows



## Feature Engineering

Creamos un solo vector con todos los features i.e 'var_1', 'var_2', 'var_3', 'var_4', 'var_5', a este le llamaremos "features" y como salida colocamos a 'label':

In [7]:
# Vector Ensamblador

# VectorAssembler" para crear un único vector de features y clase, para ser usado por nuestro modelo de entrenamiento
from pyspark.ml.feature import VectorAssembler

# Construimos el vector:
df_assembler = VectorAssembler(inputCols=['var_1',
 'var_2',
 'var_3',
 'var_4',
 'var_5'], outputCol="features")

df = df_assembler.transform(df) 

In [8]:
# visulizacion de vector ensamblado compuesto por features y label

df.select(['features','label']).show(20,False)

+------------------------------+-----+
|features                      |label|
+------------------------------+-----+
|[734.0,688.0,81.0,0.328,0.259]|0.418|
|[700.0,600.0,94.0,0.32,0.247] |0.389|
|[712.0,705.0,93.0,0.311,0.247]|0.417|
|[734.0,806.0,69.0,0.315,0.26] |0.415|
|[613.0,759.0,61.0,0.302,0.24] |0.378|
|[748.0,676.0,85.0,0.318,0.255]|0.422|
|[669.0,588.0,97.0,0.315,0.251]|0.411|
|[667.0,845.0,68.0,0.324,0.251]|0.381|
|[758.0,890.0,64.0,0.33,0.274] |0.436|
|[726.0,670.0,88.0,0.335,0.268]|0.422|
|[583.0,794.0,55.0,0.302,0.236]|0.371|
|[676.0,746.0,72.0,0.317,0.265]|0.4  |
|[767.0,699.0,89.0,0.332,0.274]|0.433|
|[637.0,597.0,86.0,0.317,0.252]|0.374|
|[609.0,724.0,69.0,0.308,0.244]|0.382|
|[776.0,733.0,83.0,0.325,0.259]|0.437|
|[701.0,832.0,66.0,0.325,0.26] |0.39 |
|[650.0,709.0,74.0,0.316,0.249]|0.386|
|[804.0,668.0,95.0,0.337,0.265]|0.453|
|[713.0,614.0,94.0,0.31,0.238] |0.404|
+------------------------------+-----+
only showing top 20 rows



Partimos a continuación el set de datos en 75% training y 25% testing:

In [9]:
# Particion del data set

#select data for building model
model_df=df.select(['features','label'])

#split the data 
train,test = model_df.randomSplit([0.75,0.25])

print(f"Size of train Dataset : {train.count()}" )
print(f"Size of test Dataset : {test.count()}" )

Size of train Dataset : 940
Size of test Dataset : 292


Creamos el Regresor Lineal: 

In [10]:
lr = LinearRegression()

Entrenamos el modelo de regresión lineal:

In [11]:
# Fit the model, le llamamos lr_model
lr_model = lr.fit(train)

Creamos el dataframe de prediciones (*predictions_df*) a partir del modelo de entrenamiento y el conjunto de datos test: 

In [12]:
predictions_df = lr_model.transform(test)

Visualizamos el contenido de *predictions_df*:

In [13]:
# visulizacion de predictions_df

predictions_df.show(10)

+--------------------+-----+-------------------+
|            features|label|         prediction|
+--------------------+-----+-------------------+
|[468.0,746.0,52.0...|0.329|0.31912684306685224|
|[498.0,615.0,67.0...|0.318| 0.3219503254221885|
|[516.0,504.0,86.0...|0.327|0.32802809797500543|
|[522.0,621.0,72.0...|0.317|0.32707529356836323|
|[524.0,665.0,65.0...|0.336| 0.3343022494390173|
|[531.0,734.0,55.0...| 0.34| 0.3416435922631219|
|[534.0,609.0,69.0...|0.329| 0.3265589423262677|
|[537.0,660.0,63.0...|0.326| 0.3277570850357314|
|[548.0,636.0,63.0...|0.339|0.33556632888872695|
|[548.0,770.0,60.0...|0.351| 0.3519260183061831|
+--------------------+-----+-------------------+
only showing top 10 rows



Ahora, evaluamos el modelo de Regresión Lineal, con los datos de TEST:

In [14]:
# evaluacion del modelo, le llamaremos model_predictions

model_predictions = lr_model.evaluate(test)

Imprimimos el valor de R2:

In [15]:
# valor de R2

lr_model.evaluate(test).r2

0.885314580706535

Imprimimos el valor del meanSquaredError:

In [16]:
# valor del meanSquaredError

lr_model.evaluate(test).rootMeanSquaredError

0.011390670363836527

## Regresión con Árboles de Decisión

Importamos la librería *DecisionTreeRegressor*: 

In [17]:
# import lib

from pyspark.ml.regression import DecisionTreeRegressor

Creamos el Regresor DT, le llamaremos *dec_tree*:

In [18]:
# dec_tree

dec_tree = DecisionTreeRegressor()

Entrenamos el modelo:

In [19]:
# Train model, le llamaremos dec_tree_model

dec_tree_model = dec_tree.fit(train)

Cuánto es la profundidad máxima por defecto, de este algoritmo?

R/ La **profundidad máxima por defecto de este algoritmo** es de **5**

Desplegamos las *featureImportances*:

In [20]:
dec_tree_model.featureImportances

SparseVector(5, {0: 0.9634, 1: 0.0183, 2: 0.0067, 3: 0.0017, 4: 0.0098})

Evaluamos el modelo con los datos de entrenamiento:

In [21]:
# Make predictions, le llamaremos model_predictions

model_predictions = dec_tree_model.transform(test)

In [22]:
# visualizamos

model_predictions.show()

+--------------------+-----+-------------------+
|            features|label|         prediction|
+--------------------+-----+-------------------+
|[468.0,746.0,52.0...|0.329|             0.3268|
|[498.0,615.0,67.0...|0.318|0.31853846153846155|
|[516.0,504.0,86.0...|0.327|0.31853846153846155|
|[522.0,621.0,72.0...|0.317|0.31853846153846155|
|[524.0,665.0,65.0...|0.336|0.31853846153846155|
|[531.0,734.0,55.0...| 0.34|             0.3268|
|[534.0,609.0,69.0...|0.329|0.31853846153846155|
|[537.0,660.0,63.0...|0.326|0.31853846153846155|
|[548.0,636.0,63.0...|0.339|             0.3398|
|[548.0,770.0,60.0...|0.351|              0.359|
|[556.0,675.0,67.0...|0.348|            0.35044|
|[557.0,659.0,71.0...|0.355|            0.35044|
|[558.0,688.0,67.0...| 0.35|            0.35044|
|[570.0,578.0,82.0...|0.363|            0.35044|
|[570.0,655.0,66.0...| 0.34|            0.35044|
|[572.0,646.0,71.0...|0.329|            0.35044|
|[573.0,656.0,75.0...|0.345|            0.35044|
|[573.0,860.0,54.0..

Importamos el **RegressionEvaluator**

In [23]:
# import Evaluator

from pyspark.ml.evaluation import RegressionEvaluator

Usando *RegressionEvaluator* calculamos e imprimimos el valor de las metricas R2 y RMSE:

In [24]:
# R2 value of the model on test data 
dt_evaluator = RegressionEvaluator(metricName='r2')
dt_r2 = dt_evaluator.evaluate(model_predictions)
print(f'The r-square value of DecisionTreeRegressor is {dt_r2}')
print(" ")

# RMSE value of the model on test data 
dt_evaluator = RegressionEvaluator(metricName='rmse')
dt_rmse = dt_evaluator.evaluate(model_predictions)
print(f'Root Mean Squared Error (RMSE) value of DecisionTreeRegressor is {dt_rmse}')

The r-square value of DecisionTreeRegressor is 0.851845744456401
 
Root Mean Squared Error (RMSE) value of DecisionTreeRegressor is 0.012946496130240424


## RandomForestRegressor

Importamos a *RandomForestRegressor*

In [25]:
# import lib

from pyspark.ml.regression import RandomForestRegressor

Creamos el Regresor RF:

In [26]:
# Regresor 

rf = RandomForestRegressor()

Entrenamos el modelo:

In [27]:
# Train model, le llamaremos rf_model

rf_model = rf.fit(train)

Desplegamos las *featureImportances*:

In [28]:
# importances 

rf_model.featureImportances

SparseVector(5, {0: 0.5885, 1: 0.0598, 2: 0.0365, 3: 0.2252, 4: 0.09})

Desplegamos el numero de arboles (Num of Trees)

In [29]:
# Numero de Trees

rf_model.getNumTrees

20

Evaluamos el modelo con los datos de entrenamiento, le llamaremos model_predictions:


In [30]:
# model_predictions

model_predictions = rf_model.transform(test)

Desplegamos los valores del *model_predictions*

In [31]:
model_predictions.show(10)

+--------------------+-----+-------------------+
|            features|label|         prediction|
+--------------------+-----+-------------------+
|[468.0,746.0,52.0...|0.329| 0.3326435538685171|
|[498.0,615.0,67.0...|0.318|  0.326602086843781|
|[516.0,504.0,86.0...|0.327|0.32770304983503806|
|[522.0,621.0,72.0...|0.317|  0.326602086843781|
|[524.0,665.0,65.0...|0.336| 0.3269624485653927|
|[531.0,734.0,55.0...| 0.34| 0.3326435538685171|
|[534.0,609.0,69.0...|0.329| 0.3309687309375904|
|[537.0,660.0,63.0...|0.326|0.33091132709143656|
|[548.0,636.0,63.0...|0.339|0.35288763429772196|
|[548.0,770.0,60.0...|0.351|0.34135316266193233|
+--------------------+-----+-------------------+
only showing top 10 rows



Usando *RegressionEvaluator* calculamos e imprimimos el valor de las metricas R2 y RMSE:

In [32]:
# R2 value of the model on test data 
rf_evaluator = RegressionEvaluator(metricName='r2')
rf_r2 = rf_evaluator.evaluate(model_predictions)
print(f'The r-square value of DecisionTreeRegressor is {rf_r2}')
print(" ")


# RMSE value of the model on test data 
rf_evaluator = RegressionEvaluator(metricName='rmse')
rf_rmse = rf_evaluator.evaluate(model_predictions)
print(f'The RMSE of DecisionTreeRegressor is {rf_rmse}')

The r-square value of DecisionTreeRegressor is 0.8592613278489709
 
The RMSE of DecisionTreeRegressor is 0.012618330700327328


## Gradient-Boosted Tree Regressor

Importamos a GBTRegressor


In [33]:
# import

from pyspark.ml.regression import GBTRegressor

Creamos el Regresor GBTR, le llamaremos gbt:


In [34]:
# regresor

gbt = GBTRegressor()

Entrenamos el modelo:

In [35]:
# Train model, le llamaremos gbt_model

gbt_model = gbt.fit(train)

Desplegamos las featureImportances:

In [36]:
#Importances

gbt_model.featureImportances

SparseVector(5, {0: 0.2181, 1: 0.2372, 2: 0.1637, 3: 0.1724, 4: 0.2086})

Evaluamos el modelo con los datos de entrenamiento, le llamaremos model_predictions:

In [37]:
# Model

model_predictions = gbt_model.transform(test)

Desplegamos los valores del *model_predictions*

In [38]:
# show 

model_predictions.show(10)

+--------------------+-----+-------------------+
|            features|label|         prediction|
+--------------------+-----+-------------------+
|[468.0,746.0,52.0...|0.329|0.32470001945280874|
|[498.0,615.0,67.0...|0.318|0.31669836838742976|
|[516.0,504.0,86.0...|0.327| 0.3193070071676158|
|[522.0,621.0,72.0...|0.317| 0.3224046596393666|
|[524.0,665.0,65.0...|0.336| 0.3175934249623884|
|[531.0,734.0,55.0...| 0.34| 0.3238293674956955|
|[534.0,609.0,69.0...|0.329| 0.3176848836578957|
|[537.0,660.0,63.0...|0.326| 0.3169431851107243|
|[548.0,636.0,63.0...|0.339| 0.3379508452141244|
|[548.0,770.0,60.0...|0.351|0.35829623896385104|
+--------------------+-----+-------------------+
only showing top 10 rows



Usando RegressionEvaluator calculamos e imprimimos el valor de las metricas R2 y RMSE:

In [39]:
 #Select (prediction, true label) and compute test error
    
# R2 value of the model on test data 
gbt_evaluator = RegressionEvaluator(metricName='r2')
gbt_r2 = gbt_evaluator.evaluate(model_predictions)
print(f'The r-square value of DecisionTreeRegressor is {gbt_r2}')
print(" ")


# RMSE value of the model on test data 
gbt_evaluator = RegressionEvaluator(metricName='rmse')
gbt_rmse = gbt_evaluator.evaluate(model_predictions)
print(f'The RMSE of DecisionTreeRegressor is {gbt_rmse}')

The r-square value of DecisionTreeRegressor is 0.8441596763670113
 
The RMSE of DecisionTreeRegressor is 0.013278074507811937


 ## Exploracion de datos...

Usaremos el dataset https://archive.ics.uci.edu/ml/datasets/Bank+Marketing 

Indique a grandes razgos de que se trata este dataset:


**El dataset está relacionado con campañas directas de mercadeo de una institución bancaria portuguesa. Las campañas de mercadeo se basaron en llamadas telefónicas. A menudo, se requería más de un contacto con el mismo cliente para acceder si el producto (depósito bancario a plazo) estaría ('sí') o no ('no') suscrito.**

Carga de datos, archivo bank_data.csv:


In [40]:
# Load csv Dataset

df=spark.read.csv('bank_data.csv',inferSchema=True,header=True)

Determine la cantidad de datos en el dataset:

In [41]:
#number of records

print("Cantidad de datos del DataFrame: ", df.count())
print(" ")
print("Cantidad de columnas del DataFrame: ", len(df.columns))

Cantidad de datos del DataFrame:  41188
 
Cantidad de columnas del DataFrame:  21


A que dato corresponde cada columna?

In [42]:
# columns values

df.columns

['age',
 'job',
 'marital',
 'education',
 'default',
 'housing',
 'loan',
 'contact',
 'month',
 'day_of_week',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'poutcome',
 'emp.var.rate',
 'cons.price.idx',
 'cons.conf.idx',
 'euribor3m',
 'nr.employed',
 'target_class']

Imprima el Schema:

In [43]:
#dataype of input data - Schema

df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- month: string (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- emp.var.rate: double (nullable = true)
 |-- cons.price.idx: double (nullable = true)
 |-- cons.conf.idx: double (nullable = true)
 |-- euribor3m: double (nullable = true)
 |-- nr.employed: double (nullable = true)
 |-- target_class: string (nullable = true)



En cuanto a la salida, como es la distribución de clases?

In [44]:
# YES/NO Class Distribution

df.groupBy('target_class').count().show()

+------------+-----+
|target_class|count|
+------------+-----+
|          no|36548|
|         yes| 4640|
+------------+-----+



Una tarea típica, resulta de convertir los valores binarios en 1 y 0, usando como referencia "label", convierta los no/yes en 0/1:

In [45]:
from pyspark.sql import functions as F
from pyspark.sql import *

In [46]:
# Ingrese acá la instrucción: 

df = df.withColumn("target_class", F.when(F.col("target_class") == "yes", 1).otherwise(F.col("target_class")))

df = df.withColumn("target_class", F.when(F.col("target_class") == "no", 0).otherwise(F.col("target_class")))

In [47]:
# New 1/0 Class Distribution

df.groupBy('target_class').count().show()

+------------+-----+
|target_class|count|
+------------+-----+
|           0|36548|
|           1| 4640|
+------------+-----+



A continuación se presenta un ejercicio de Deep Learning para su revisión...

# Deep Learning 

Importamos las librerias necesarias:

In [48]:
import os
import numpy as np
import pandas as pd
from pyspark.sql.types import *
from pyspark.ml import Pipeline
from pyspark.sql import functions as f
from pyspark.sql.functions import udf, StringType
from pyspark.sql import SparkSession, functions as F
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.feature import OneHotEncoder, VectorAssembler, StringIndexer

Inicializamos la sesion SPARK:

In [49]:
spark = SparkSession.builder.appName('deep_learning').getOrCreate()

Leemos el dataset:

In [50]:
data = spark.read.csv('dl_data.csv', header=True, inferSchema=True)

In [51]:
data.printSchema()

root
 |-- Visit_Number_Bucket: string (nullable = true)
 |-- Page_Views_Normalized: double (nullable = true)
 |-- Orders_Normalized: integer (nullable = true)
 |-- Internal_Search_Successful_Normalized: double (nullable = true)
 |-- Internal_Search_Null_Normalized: double (nullable = true)
 |-- Email_Signup_Normalized: double (nullable = true)
 |-- Total_Seconds_Spent_Normalized: double (nullable = true)
 |-- Store_Locator_Search_Normalized: double (nullable = true)
 |-- Mapped_Last_Touch_Channel: string (nullable = true)
 |-- Mapped_Mobile_Device_Type: string (nullable = true)
 |-- Mapped_Browser_Type: string (nullable = true)
 |-- Mapped_Entry_Pages: string (nullable = true)
 |-- Mapped_Site_Section: string (nullable = true)
 |-- Mapped_Promo_Code: string (nullable = true)
 |-- Maped_Product_Name: string (nullable = true)
 |-- Mapped_Search_Term: string (nullable = true)
 |-- Mapped_Product_Collection: string (nullable = true)



Renombramos la columna TARGET:

In [52]:
data = data.withColumnRenamed('Orders_Normalized', 'label')

In [53]:
data.printSchema()

root
 |-- Visit_Number_Bucket: string (nullable = true)
 |-- Page_Views_Normalized: double (nullable = true)
 |-- label: integer (nullable = true)
 |-- Internal_Search_Successful_Normalized: double (nullable = true)
 |-- Internal_Search_Null_Normalized: double (nullable = true)
 |-- Email_Signup_Normalized: double (nullable = true)
 |-- Total_Seconds_Spent_Normalized: double (nullable = true)
 |-- Store_Locator_Search_Normalized: double (nullable = true)
 |-- Mapped_Last_Touch_Channel: string (nullable = true)
 |-- Mapped_Mobile_Device_Type: string (nullable = true)
 |-- Mapped_Browser_Type: string (nullable = true)
 |-- Mapped_Entry_Pages: string (nullable = true)
 |-- Mapped_Site_Section: string (nullable = true)
 |-- Mapped_Promo_Code: string (nullable = true)
 |-- Maped_Product_Name: string (nullable = true)
 |-- Mapped_Search_Term: string (nullable = true)
 |-- Mapped_Product_Collection: string (nullable = true)



Partimos lo datos en Train, Validation y Test:

In [54]:
train, validation, test  = data.randomSplit([0.7, 0.2, 0.1], 1234)

Construimos el Pipeline

In [55]:
categorical_columns = [item[0] for item in data.dtypes if item[1].startswith('string')]
numeric_columns = [item[0] for item in data.dtypes if item[1].startswith('double')]

indexers = [StringIndexer(inputCol=column, outputCol='{0}_index'.format(column)) for column in categorical_columns]

featuresCreator = VectorAssembler(inputCols=[indexer.getOutputCol() for indexer in indexers] + numeric_columns, outputCol="features")

layers = [len(featuresCreator.getInputCols()), 4, 2, 2]

classifier = MultilayerPerceptronClassifier(labelCol='label', featuresCol='features', maxIter=100, layers=layers, blockSize=128, seed=1234)

pipeline = Pipeline(stages=indexers + [featuresCreator, classifier])

Entrenamos...

In [56]:
model = pipeline.fit(train)

Validamos y Evaluamos

In [57]:
train_output_df = model.transform(train)
validation_output_df = model.transform(validation)
test_output_df = model.transform(test)

Llevamos a cabo, algunas predicciones:

In [58]:
train_predictionAndLabels = train_output_df.select("prediction", "label")
validation_predictionAndLabels = validation_output_df.select("prediction", "label")
test_predictionAndLabels = test_output_df.select("prediction", "label")

metrics = ['weightedPrecision', 'weightedRecall', 'accuracy']

for metric in metrics:
    evaluator = MulticlassClassificationEvaluator(metricName=metric)
    print('Train ' + metric + ' = ' + str(evaluator.evaluate(train_predictionAndLabels)))
    print('Validation ' + metric + ' = ' + str(evaluator.evaluate(validation_predictionAndLabels)))
    print('Test ' + metric + ' = ' + str(evaluator.evaluate(test_predictionAndLabels)))

Train weightedPrecision = 0.9700274536972262
Validation weightedPrecision = 0.9710550450398906
Test weightedPrecision = 0.9728431235390542
Train weightedRecall = 0.9695765649955203
Validation weightedRecall = 0.9705284552845528
Test weightedRecall = 0.9725118483412322
Train accuracy = 0.9695765649955203
Validation accuracy = 0.9705284552845529
Test accuracy = 0.9725118483412323


Puede mejorar el test accuracy del modelo variando alguno de los hyperparametros?

In [59]:
# Partimos lo datos en Train, Validation y Test:
#train, validation, test  = data.randomSplit([0.7, 0.3, 0.1], 1234)

# Construimos el Pipeline
categorical_columns = [item[0] for item in data.dtypes if item[1].startswith('string')]
numeric_columns = [item[0] for item in data.dtypes if item[1].startswith('double')]

indexers = [StringIndexer(inputCol=column, outputCol='{0}_index'.format(column)) for column in categorical_columns]

featuresCreator = VectorAssembler(inputCols=[indexer.getOutputCol() for indexer in indexers] + numeric_columns, outputCol="features")

#Cambio en layer intermedia: 2 --> 6
layers = [len(featuresCreator.getInputCols()), 4, 6, 2]

classifier = MultilayerPerceptronClassifier(labelCol='label', featuresCol='features', maxIter=100, layers=layers, blockSize=128, seed=1234)

pipeline = Pipeline(stages=indexers + [featuresCreator, classifier])

# Entrenamos...
model = pipeline.fit(train)

# Validamos y Evaluamos
train_output_df = model.transform(train)
validation_output_df = model.transform(validation)
test_output_df = model.transform(test)

# Llevamos a cabo, algunas predicciones:
train_predictionAndLabels = train_output_df.select("prediction", "label")
validation_predictionAndLabels = validation_output_df.select("prediction", "label")
test_predictionAndLabels = test_output_df.select("prediction", "label")

metrics = ['weightedPrecision', 'weightedRecall', 'accuracy']

for metric in metrics:
    evaluator = MulticlassClassificationEvaluator(metricName=metric)
    print('Train ' + metric + ' = ' + str(evaluator.evaluate(train_predictionAndLabels)))
    print('Validation ' + metric + ' = ' + str(evaluator.evaluate(validation_predictionAndLabels)))
    print('Test ' + metric + ' = ' + str(evaluator.evaluate(test_predictionAndLabels)))

Train weightedPrecision = 0.9793181237737344
Validation weightedPrecision = 0.9794162359121685
Test weightedPrecision = 0.9824994403242441
Train weightedRecall = 0.9790619765494137
Validation weightedRecall = 0.9791327913279133
Test weightedRecall = 0.9822613405551794
Train accuracy = 0.9790619765494137
Validation accuracy = 0.9791327913279133
Test accuracy = 0.9822613405551794
